# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!

- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/arnabbhattacharya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/arnabbhattacharya/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [4]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Use-Case Data!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [5]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()  # loaded all the docs in the data folder, each page is a doc

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [6]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [ ]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [ ]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [ ]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

We can save and load our knowledge graphs as follows.

In [ ]:
# save the knowledge graph
kg.save("usecase_data_kg.json")

In [ ]:
usecase_data_kg = KnowledgeGraph.load("usecase_data_kg.json")
usecase_data_kg

KnowledgeGraph(nodes: 86, relationships: 712)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [13]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=usecase_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [17]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.

✅ **Answer**: Query synthesizers are specialized RAGAS components that can automatically generate test questions for evaluating RAG applications. Specifically, they work by analyzing the knowledge graph to find relevant nodes and relationships, create personas (fictional users that interact with your data), generate scenarios that combine personas with specific data contexts and finally produce realistic questions that these personas would ask. Next, we describe each query synthesizer:

1. *SingleHopSpecificQuerySynthesizer* -  creates simple, direct questions that can be answered from a single piece of information. It finds nodes with named entities, creates personas who would be interested in those entitiies and generates starightforward questions. e.g. "Which AI companies are mentioned in this document?"

2. *MultiHopAbstractQuerySynthesizer* - creates complex questions requiring reasoning across multiple related concepts and themes. It works by clustering nodes based on summary similarity, extracts abstract themes from multiple documents, generates personas interested in broader patterns and finally combining them to create conceptual questions. e.g. "How do the themes of AI adoption in healthcare relate to patient outcome improvements across different demographic groups?"

3. *MultiHopSpecificQuerySynthesizer* - creates questions that require connecting specific facts across multiple documents through shared entities. It works by finding pairs of nodes that shares common entities, identifies overlapping info between documents, creates personas interested in specific factual connections, and generates questions requiring factual reasoning. e.g. "What specific healthcare applications has Dr. Smith developed using machine learning?"



Finally, we can use our `TestSetGenerator` to generate our testset!

In [18]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/10 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,"How has ChatGPT, launched in November 2022, ac...",[Introduction ChatGPT launched in November 202...,"ChatGPT, introduced in November 2022, experien...",single_hop_specifc_query_synthesizer
1,Who Handa et al 2025,[Table 1: ChatGPT daily message counts (millio...,The context mentions Handa et al. (2025) in re...,single_hop_specifc_query_synthesizer
2,How does a Secton in the context of occupation...,[Variation by Occupation Figure 23 presents va...,Variation by Occupation Figure 23 presents var...,single_hop_specifc_query_synthesizer
3,What practical guidance does the paper provide...,[Conclusion This paper studies the rapid growt...,The paper documents eight important facts abou...,single_hop_specifc_query_synthesizer
4,how AI applications in work and non work conte...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,"The context explains that ChatGPT, launched in...",multi_hop_abstract_query_synthesizer
5,H0w does the varriation in ChatGPT usag by ocu...,[<1-hop>\n\nVariation by Occupation Figure 23 ...,The context shows that users in highly paid pr...,multi_hop_abstract_query_synthesizer
6,How do large language models like ChatGPT impa...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,"The context indicates that ChatGPT, based on l...",multi_hop_abstract_query_synthesizer
7,How do the classifications in Appendix B and A...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,The classifications detailed in Appendix B and...,multi_hop_specific_query_synthesizer
8,How does the rapid growth of ChatGPT usage in ...,[<1-hop>\n\nTable 1: ChatGPT daily message cou...,"The rapid growth of ChatGPT usage in the US, w...",multi_hop_specific_query_synthesizer
9,how Claude use ChatGPT for work and Claude,[<1-hop>\n\nTable 1: ChatGPT daily message cou...,The context shows that ChatGPT is used for var...,multi_hop_specific_query_synthesizer


In [23]:
import pandas as pd

# save the testset 
#pd.DataFrame(testset.to_pandas()).to_csv("testset.csv", index=False)

#read testset
testset = pd.read_csv("testset.csv")
testset


,user_input,reference_contexts,reference,synthesizer_name
0,"How has ChatGPT, launched in November 2022, ac...",['Introduction ChatGPT launched in November 20...,"ChatGPT, introduced in November 2022, experien...",single_hop_specifc_query_synthesizer
1,Who Handa et al 2025,['Table 1: ChatGPT daily message counts (milli...,The context mentions Handa et al. (2025) in re...,single_hop_specifc_query_synthesizer
2,How does a Secton in the context of occupation...,['Variation by Occupation Figure 23 presents v...,Variation by Occupation Figure 23 presents var...,single_hop_specifc_query_synthesizer
3,What practical guidance does the paper provide...,['Conclusion This paper studies the rapid grow...,The paper documents eight important facts abou...,single_hop_specifc_query_synthesizer
4,how AI applications in work and non work conte...,['<1-hop>\n\nIntroduction ChatGPT launched in ...,"The context explains that ChatGPT, launched in...",multi_hop_abstract_query_synthesizer
5,H0w does the varriation in ChatGPT usag by ocu...,['<1-hop>\n\nVariation by Occupation Figure 23...,The context shows that users in highly paid pr...,multi_hop_abstract_query_synthesizer
6,How do large language models like ChatGPT impa...,['<1-hop>\n\nIntroduction ChatGPT launched in ...,"The context indicates that ChatGPT, based on l...",multi_hop_abstract_query_synthesizer
7,How do the classifications in Appendix B and A...,['<1-hop>\n\nIntroduction ChatGPT launched in ...,The classifications detailed in Appendix B and...,multi_hop_specific_query_synthesizer
8,How does the rapid growth of ChatGPT usage in ...,['<1-hop>\n\nTable 1: ChatGPT daily message co...,"The rapid growth of ChatGPT usage in the US, w...",multi_hop_specific_query_synthesizer
9,how Claude use ChatGPT for work and Claude,['<1-hop>\n\nTable 1: ChatGPT daily message co...,The context shows that ChatGPT is used for var...,multi_hop_specific_query_synthesizer


In [36]:
# just printing out row for easier viewing
row = 5

print(f"Synthesizer used: {testset.iloc[row]['synthesizer_name']}")
print(f"\n \nUser query: {testset.iloc[row]['user_input']}")
print(f"\n \n Reference output: {testset.iloc[row]['reference']}")
print(f"\n \n Reference contexts: {testset.iloc[row]['reference_contexts']}")

Synthesizer used: multi_hop_abstract_query_synthesizer

 
User query: H0w does the varriation in ChatGPT usag by ocupation impact the impackt of ocupation on ChatGPT usag, especilly in high-payd proffesional jobs like sciense and tech?

 
 Reference output: The context shows that users in highly paid professional and technical occupations, such as those in science and technology, are more likely to use ChatGPT for work-related tasks, especially for asking questions and technical help. The variation in usage by occupation indicates that work-related messages are focused on core job tasks, with scientific and technical occupations showing a higher tendency to use ChatGPT for asking questions, while management and business occupations focus more on writing. This demonstrates that the impact of occupation on ChatGPT usage is significant, with higher-paid professional roles engaging more in specific types of interactions.

 
 Reference contexts: ['<1-hop>\n\nVariation by Occupation Figure 2

### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [40]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/21 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/64 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to ap

Applying SummaryExtractor:   0%|          | 0/38 [00:00<?, ?it/s]

Property 'summary' already exists in node 'ac6d45'. Skipping!
Property 'summary' already exists in node 'ecbb6f'. Skipping!
Property 'summary' already exists in node 'd64c9d'. Skipping!
Property 'summary' already exists in node 'cf3028'. Skipping!
Property 'summary' already exists in node 'c44e96'. Skipping!
Property 'summary' already exists in node '8ea4fd'. Skipping!
Property 'summary' already exists in node 'bc894b'. Skipping!
Property 'summary' already exists in node '15885d'. Skipping!
Property 'summary' already exists in node 'e6c4d9'. Skipping!
Property 'summary' already exists in node 'dc8d6a'. Skipping!
Property 'summary' already exists in node 'fa5b8f'. Skipping!
Property 'summary' already exists in node '62bfe8'. Skipping!
Property 'summary' already exists in node 'd4b114'. Skipping!
Property 'summary' already exists in node 'd7fe5f'. Skipping!
Property 'summary' already exists in node '63d0e5'. Skipping!
Property 'summary' already exists in node 'ec7468'. Skipping!
Property

...

Property 'summary_embedding' already exists in node 'cf3028'. Skipping!
Property 'summary_embedding' already exists in node 'ecbb6f'. Skipping!
Property 'summary_embedding' already exists in node 'ac6d45'. Skipping!
Property 'summary_embedding' already exists in node '8ea4fd'. Skipping!
Property 'summary_embedding' already exists in node 'bc894b'. Skipping!
Property 'summary_embedding' already exists in node 'd64c9d'. Skipping!
Property 'summary_embedding' already exists in node '15885d'. Skipping!
Property 'summary_embedding' already exists in node 'c44e96'. Skipping!
Property 'summary_embedding' already exists in node 'd4b114'. Skipping!
Property 'summary_embedding' already exists in node 'e6c4d9'. Skipping!
Property 'summary_embedding' already exists in node 'dc8d6a'. Skipping!
Property 'summary_embedding' already exists in node '62bfe8'. Skipping!
Property 'summary_embedding' already exists in node 'fa5b8f'. Skipping!
Property 'summary_embedding' already exists in node '63d0e5'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [41]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,"How many users are using ChatGPT, totaling 700...",[Introduction ChatGPT launched in November 202...,"By July 2025, 700 million users were sending a...",single_hop_specifc_query_synthesizer
1,What significance does June 2024 hold in the c...,[Table 1: ChatGPT daily message counts (millio...,June 2024 is referenced as the end date for th...,single_hop_specifc_query_synthesizer
2,What are the SOC2 codes 11 in relation to SOC2...,[Variation by Occupation Figure 23 presents va...,Variation by Occupation Figure 23 presents var...,single_hop_specifc_query_synthesizer
3,How does the data from July 2025 inform our un...,[Conclusion This paper studies the rapid growt...,The paper documents that no user messages were...,single_hop_specifc_query_synthesizer
4,How do types of user queries and topics relate...,[<1-hop>\n\nConclusion This paper studies the ...,The context indicates that ChatGPT has experie...,multi_hop_abstract_query_synthesizer
5,How do types of user queries and topics relate...,[<1-hop>\n\nConclusion This paper studies the ...,The context indicates that about 70% of ChatGP...,multi_hop_abstract_query_synthesizer
6,"ChatGPT use by job, privacy data agg, how does...",[<1-hop>\n\nVariation by Occupation Figure 23 ...,The context shows variation in ChatGPT usage b...,multi_hop_abstract_query_synthesizer
7,how growth in chatbot message volume and diffe...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,"The rapid growth in chatbot message volume, re...",multi_hop_abstract_query_synthesizer
8,how do 700 million users use chatgpt for work ...,[<1-hop>\n\nConclusion This paper studies the ...,"According to the context, by July 2025, ChatGP...",multi_hop_specific_query_synthesizer
9,How does the data from November 2022 and Novem...,[<1-hop>\n\nConclusion This paper studies the ...,The context indicates that ChatGPT was launche...,multi_hop_specific_query_synthesizer


In [ ]:
# store this dataset 
#pd.DataFrame(dataset.to_pandas()).to_csv("dataset.csv", index=False)

# read the dataset (as needed)
dataset = pd.read_csv("dataset.csv")
dataset


,user_input,reference_contexts,reference,synthesizer_name
0,"How many users are using ChatGPT, totaling 700...",['Introduction ChatGPT launched in November 20...,"By July 2025, 700 million users were sending a...",single_hop_specifc_query_synthesizer
1,What significance does June 2024 hold in the c...,['Table 1: ChatGPT daily message counts (milli...,June 2024 is referenced as the end date for th...,single_hop_specifc_query_synthesizer
2,What are the SOC2 codes 11 in relation to SOC2...,['Variation by Occupation Figure 23 presents v...,Variation by Occupation Figure 23 presents var...,single_hop_specifc_query_synthesizer
3,How does the data from July 2025 inform our un...,['Conclusion This paper studies the rapid grow...,The paper documents that no user messages were...,single_hop_specifc_query_synthesizer
4,How do types of user queries and topics relate...,['<1-hop>\n\nConclusion This paper studies the...,The context indicates that ChatGPT has experie...,multi_hop_abstract_query_synthesizer
5,How do types of user queries and topics relate...,['<1-hop>\n\nConclusion This paper studies the...,The context indicates that about 70% of ChatGP...,multi_hop_abstract_query_synthesizer
6,"ChatGPT use by job, privacy data agg, how does...",['<1-hop>\n\nVariation by Occupation Figure 23...,The context shows variation in ChatGPT usage b...,multi_hop_abstract_query_synthesizer
7,how growth in chatbot message volume and diffe...,['<1-hop>\n\nIntroduction ChatGPT launched in ...,"The rapid growth in chatbot message volume, re...",multi_hop_abstract_query_synthesizer
8,how do 700 million users use chatgpt for work ...,['<1-hop>\n\nConclusion This paper studies the...,"According to the context, by July 2025, ChatGP...",multi_hop_specific_query_synthesizer
9,How does the data from November 2022 and Novem...,['<1-hop>\n\nConclusion This paper studies the...,The context indicates that ChatGPT was launche...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [43]:
from langsmith import Client

client = Client()

dataset_name = "Use Case Synthetic Data - AIE8"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Synthetic Data for Use Cases"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [49]:
for data_row in dataset.iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [50]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [51]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [52]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [53]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Use Case RAG"
)

In [54]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [56]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

As is usual: We'll be using `gpt-4.1-mini` for our RAG!

In [57]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [59]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [60]:
rag_chain.invoke({"question" : "What are people doing with AI these days?"})

'Based on the provided context, people are using AI, particularly generative AI like ChatGPT, in a variety of ways both at work and outside of work. Key activities include:\n\n- Performing workplace tasks by either augmenting or automating human labor.\n- Producing writing, software code, spreadsheets, and other digital products, which distinguishes generative AI from traditional web search engines.\n- Seeking information and advice.\n- Using AI agents as co-workers that produce output or as co-pilots that give advice and improve human problem-solving productivity.\n- Engaging in activities related to self-expression such as relationships, personal reflection, games, and role play, though these are smaller in proportion.\n\nOverall, people use AI flexibly to support professional tasks, creative endeavors, and personal expression.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [61]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [62]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dopeness_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this response dope, lit, cool, or is it just a generic response?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`: uses LLM to directly evaluate the answer correctness for a given question/query, i.e. it checks if the answer is factually correct and relevant to what is asked. It does not use any ground-truth or reference data.
- `labeled_helpfulness_evaluator`: this is a criteria-based evaluator that evaluates how helpful an answer is to a given query. Here the evaluator LLM uses reference answers to compare against.
- `dopeness_evaluator`: this is another criteria-based evaluator that evaluates if the response is dope (matching gen-Z or gen-X language patterns). Here the evaluator does not use any reference data and relies on LLM's judgement to evaluate the answer based on dopness criteria.

There are many additional ```EvaluatorType```s in LangChain that is useful if you need to check additional properties:
https://python.langchain.com/api_reference/langchain/evaluation/langchain.evaluation.schema.EvaluatorType.html#langchain.evaluation.schema.EvaluatorType

## LangSmith Evaluation

In [63]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dopeness_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'respectful-dirt-88' at:
https://smith.langchain.com/o/08071020-873b-4614-a512-80a2bbd38f89/datasets/4c6f49ea-fef9-48b1-9bc3-0a98da487d8f/compare?selectedSessions=66105f91-996f-4983-9320-edecc84c02b1




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,US ChatGPT usage data shows more non-work mess...,Based on the provided context:\n\n- About 70% ...,None,"The context indicates that by July 2025, over ...",1,1,0,6.964668,4a98e0ac-0d35-406c-bebc-e4ad1100700c,d6d7b7b1-ed25-4b23-80f5-33505faa2b9c
1,How US usage of ChatGPT in 2025 shows the impa...,"Based on the provided context, US usage of Cha...",None,"The context indicates that by July 2025, US Ch...",1,1,0,2.982058,fb92f19b-4782-405a-8961-4618d9590360,92f9db70-ef05-4133-94cf-5e3d0c8a3072
2,How does the data from November 2022 and Novem...,The data from November 2022 to July 2025 refle...,None,The context indicates that ChatGPT was launche...,1,1,0,4.949526,560c65d4-4c2b-434a-8ab6-93bad9c3e8d3,9715cd80-dd18-4c24-b15f-614c133065da
3,how do 700 million users use chatgpt for work ...,"Based on the provided context, ChatGPT users b...",None,"According to the context, by July 2025, ChatGP...",1,1,0,10.307485,3ab2ef37-4fd5-41f9-8679-a5102ea176f6,fcd4c221-f18e-41ea-835a-e8d59b55b012
4,how growth in chatbot message volume and diffe...,The context indicates that the study on ChatGP...,None,"The rapid growth in chatbot message volume, re...",1,1,0,5.013336,75ed5ebb-8ef3-4ed5-90da-97a0c03bb01b,b3093f73-87c3-4eae-b157-99a94d1b1bb4
5,"ChatGPT use by job, privacy data agg, how does...",The paper uses a secure data clean room protoc...,None,The context shows variation in ChatGPT usage b...,1,1,0,1.841999,25ce1ef4-5c8a-4d9e-a7be-86ab7d5f7e31,cc9d9743-d00f-4f65-bb17-11f3d38efde1
6,How do types of user queries and topics relate...,"Based on the provided context, user queries on...",None,The context indicates that about 70% of ChatGP...,1,1,0,10.731982,9a2ed9fa-30b7-4c58-90bf-634deca2ddd7,7a23fa00-95f1-4e73-b1dd-c53e8636d409
7,How do types of user queries and topics relate...,I don't know.,None,The context indicates that ChatGPT has experie...,0,0,0,0.645505,46d074dd-4e0d-4439-b945-9bad5dea8cd2,2701c11a-fd80-4e0a-a4d9-a7945ec576ca
8,How does the data from July 2025 inform our un...,"Based on the provided context, the data from J...",None,The paper documents that no user messages were...,1,1,0,3.172047,d4eaf879-4ab6-4b96-afcf-27f93a1ff4cf,2c0d15bb-a601-440a-9304-9f5756ac37dc
9,What are the SOC2 codes 11 in relation to SOC2...,SOC2 code 11 corresponds to Management and bus...,None,Variation by Occupation Figure 23 presents var...,0,1,0,0.909655,f9355486-1276-4a4b-9675-6ad506216765,7829b771-2dfb-49b1-bad9-e4fed45e9779


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [65]:
DOPENESS_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Make your answer rad, ensure high levels of dopeness. Do not be generic, or give generic responses.

Context: {context}
Question: {question}
"""

dopeness_rag_prompt = ChatPromptTemplate.from_template(DOPENESS_RAG_PROMPT)

In [66]:
rag_documents = docs

In [67]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

✅ **Answer**: Larger chunk sizes can help in the following ways:

- Capture more context and complete thoughts across chunk boundaries (although it may contain more irrelevant info)
- Better for complex for complex reasoning questions requiring multiple pieces of information from the same document and thereby provide more complete answers - less likely to split important information mid-thought

Here performance is strictly defined in terms of "quality" of the generated answers, but we should also keep in mind that larger chunks may have more irrelevant context info, and may hit model context token limits that hurts test-time execution. Lot of times, the answer depends on the document structure, content, and identifying relevant context to retrieve (e.g., re-ranking algorithms) 

In [74]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

✅ **Answer**: Better (or larger embedding models) allow for more richer representations of the domain text that can
- better and more accurate semantic understanding (both queries and context);
- improve context awareness resulting in better understanding of document meaning and user intent;
- better at finding relevant documents even when exact keywords dont match

The key concept here is that larger embedding models can help improve "semantic similarity" that is especially critical in the retrieval process.



In [69]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Use Case RAG Docs"
)

In [70]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [71]:
dopeness_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dopeness_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [72]:
dopeness_rag_chain.invoke({"question" : "How are people using AI to make money?"})

'Alright, strap in — here’s the ultra-slick lowdown on how folks are stacking paper with AI, straight from the data vault:\n\nPeople aren’t just using AI to crank out tasks robotically; instead, they’re dialing up ChatGPT as their *advisor* and *research sidekick* — a backstage genius boosting decision-making quality. Especially in brain-heavy, knowledge-packed jobs, AI is flipping productivity switches by leveling up how choices get made. So it’s not just about automation, it’s about turbocharging human smarts to make sharper, faster calls that pay off.\n\nCollis and Brynjolfsson (2025) drop a mind-melting stat: in the US, users would pay nearly a hundred bucks *per month* just to keep this AI magic—meaning the surplus value added is a jaw-dropping $97 billion a year. That’s straight-up economic fire — workers getting paid by AI-powered decisions and insights, not just sweat and grind.\n\nIn sum: humans are cashing in by leaning on AI as a strategic partner to supercharge their hustle

Finally, we can evaluate the new chain on the same test set!

In [73]:
evaluate(
    dopeness_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dopeness_evaluator
    ],
    metadata={"revision_id": "dopeness_rag_chain"},
)

View the evaluation results for experiment: 'crazy-weather-38' at:
https://smith.langchain.com/o/08071020-873b-4614-a512-80a2bbd38f89/datasets/4c6f49ea-fef9-48b1-9bc3-0a98da487d8f/compare?selectedSessions=b134d65d-5852-48d5-bc27-71420c984ceb




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,US ChatGPT usage data shows more non-work mess...,"Yo, let’s vibe with the facts dropped in the c...",None,"The context indicates that by July 2025, over ...",0,0,1,4.678268,4a98e0ac-0d35-406c-bebc-e4ad1100700c,e9e32947-919e-4ba0-b783-e88568bb4390
1,How US usage of ChatGPT in 2025 shows the impa...,"Alright, let’s unpack this with some serious A...",None,"The context indicates that by July 2025, US Ch...",1,1,1,6.956342,fb92f19b-4782-405a-8961-4618d9590360,e4a0e955-109c-4027-8a1a-0410f319da55
2,How does the data from November 2022 and Novem...,"Yo, buckle up for this turbocharged growth sag...",None,The context indicates that ChatGPT was launche...,1,1,1,6.629185,560c65d4-4c2b-434a-8ab6-93bad9c3e8d3,7cb7c35d-71fb-4595-9cd1-75a14f0724e1
3,how do 700 million users use chatgpt for work ...,"Alright, strap in for this mega breakdown on h...",None,"According to the context, by July 2025, ChatGP...",1,1,1,8.584020,3ab2ef37-4fd5-41f9-8679-a5102ea176f6,ff65bd8b-bdea-4d47-b449-8477834671c2
4,how growth in chatbot message volume and diffe...,"Alright, buckle up — here comes the lowdown on...",None,"The rapid growth in chatbot message volume, re...",1,1,1,6.646858,75ed5ebb-8ef3-4ed5-90da-97a0c03bb01b,acff2986-102d-40b7-99fe-98e3e7eed26f
5,"ChatGPT use by job, privacy data agg, how does...","Alright, let’s blast off into the cosmos of Ch...",None,The context shows variation in ChatGPT usage b...,1,1,1,5.325604,25ce1ef4-5c8a-4d9e-a7be-86ab7d5f7e31,75037db7-b2ab-4608-8c9e-398a90cfebfc
6,How do types of user queries and topics relate...,"Alright, here comes the turbo-charged, knowled...",None,The context indicates that about 70% of ChatGP...,1,1,1,6.439466,9a2ed9fa-30b7-4c58-90bf-634deca2ddd7,60efe276-aac7-4d7c-ac1a-57d81089ba57
7,How do types of user queries and topics relate...,"Oh, this question is dripping with that next-l...",None,The context indicates that ChatGPT has experie...,1,1,1,6.391045,46d074dd-4e0d-4439-b945-9bad5dea8cd2,b532b61c-ce4b-45f6-86a9-54c5bf90e7af
8,How does the data from July 2025 inform our un...,"Alright, let’s crank this up to eleven. By Jul...",None,The paper documents that no user messages were...,1,1,1,5.802618,d4eaf879-4ab6-4b96-afcf-27f93a1ff4cf,093ae52b-67b2-40a5-b00e-0b20ffd7067c
9,What are the SOC2 codes 11 in relation to SOC2...,"Alright, let's crank up the dopeness on this h...",None,Variation by Occupation Figure 23 presents var...,1,1,1,2.436193,f9355486-1276-4a4b-9675-6ad506216765,cf143773-62e9-4779-a9d2-950a033c9515


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

![Screenshot](screenshot.png)

✅ **Answer**: From the above screenshot, we see all the 3 metrics improved for the second experiment compared to the first one. In my opinion, the dopeness metric improved dramatically because of the direct prompt enhancement made in `DOPENESS_RAG_PROMPT` that explicitly asks for "rad" answers with "high levels of dopeness" and not to give "generic answers" - things that improves the criteria mentioned in the `dopeness_evaluator`. The other 2 metrics (qa and labeled helpfulness) improved primaeily due to the underlying improvements in chunk size and embeddings that made responses more accurate and helpful overall. Its important to note that the absolute values of these metrics dont matter, rather the relative changes is indicative of whether changes to our RAG app improve quality of queries generated (in line with metrics drive development ).